In [2]:
import torch

device = torch.device('cuda')  # Ép buộc chỉ dùng GPU
print(f"Đang sử dụng thiết bị: {device}, GPU: {torch.cuda.get_device_name(0)}")

Đang sử dụng thiết bị: cuda, GPU: Tesla T4


In [3]:
!git clone https://github.com/vanminh1878/DS312-medblip_large.git

Cloning into 'DS312-medblip_large'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 74 (delta 18), reused 20 (delta 16), pack-reused 49 (from 2)
Receiving objects: 100% (74/74), 39.84 KiB | 1.73 MiB/s, done.
Resolving deltas: 100% (37/37), done.


In [4]:
cd DS312-medblip_large

/kaggle/working/DS312-medblip_large


In [1]:
!cp /kaggle/input/oggy-ds312/medblip_large_step_40046/medblip_large_step_40046.pth /kaggle/working/

<h1>Train</h1>

In [5]:
# !python main.py train --root_path /kaggle/input/if-u-know-u-know/ --batch_size 4 --num_epochs 1 --lr 1e-5 --load_weights True

2025-05-15 19:32:05.410214: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747337525.609086      89 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747337525.665373      89 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Đang sử dụng thiết bị: cuda
preprocessor_config.json: 100%|████████████████| 445/445 [00:00<00:00, 4.30MB/s]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
toke

<h1>Lưu file</h1>

In [ ]:
# import shutil

# # Đường dẫn đến tệp checkpoint
# checkpoint_path = '/kaggle/working/medblip_large_step_20023.pth'

# # Tạo tệp zip từ tệp checkpoint
# shutil.make_archive('medblip_large_step_20023', 'zip', '/kaggle/working', 'medblip_large_step_20023.pth')

In [ ]:
# import shutil

# # Đường dẫn của tệp zip gốc
# source_path = '/kaggle/working/DS312-medblip_large/medblip_large_step_20023.zip'

# # Đường dẫn đích nơi bạn muốn sao chép tệp
# destination_path = '/kaggle/working/medblip_large_step_20023.zip'

# # Sao chép tệp
# shutil.copy(source_path, destination_path)

In [ ]:
# from IPython.display import FileLink
# FileLink(r'medblip_large_step_20023.zip')

<h1>infer</h1>

In [5]:
#!python main.py predict --root_path /kaggle/input/oggy-ds312/ --path_weights /kaggle/working/

2025-05-16 08:55:51.368786: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747385751.794403      89 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747385751.903745      89 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
preprocessor_config.json: 100%|████████████████| 445/445 [00:00<00:00, 3.73MB/s]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
tokenizer_config.json: 100%|████

In [ ]:
# Cell 3: Hậu xử lý file run.csv
import pandas as pd
import re

def clean_caption(caption):
    """
    Làm sạch và chuẩn hóa caption y tế.
    
    Args:
        caption (str): Caption cần làm sạch.
        
    Returns:
        str: Caption đã được làm sạch và chuẩn hóa.
    """
    # 1. Loại bỏ các phần không hoàn chỉnh hoặc thông tin dư thừa
    caption = caption.strip()

    # 2. Chuẩn hóa khoảng trắng và dấu gạch nối
    caption = re.sub(r'\s+', ' ', caption)  # Chuẩn hóa khoảng trắng
    caption = re.sub(r'\s*-\s*', '-', caption)  # Chuẩn hóa dấu gạch nối (ví dụ: "right - sided" → "right-sided")

    # 3. Chuẩn hóa thuật ngữ y tế
    medical_terms = {
        'x-ray': 'X-ray',
        'mri': 'MRI',
        't2': 'T2',
        't1': 'T1',
        'ct': 'CT',
        'pet/ct': 'PET/CT',
        'usg': 'USG',
        '18f-fluorodeoxyglucose': '18F-fluorodeoxyglucose',
        'pet': 'PET',
        'mrcp': 'MRCP',
        'cta': 'CTA',
        'lvot': 'LVOT',
        'ao': 'AO',
        'la': 'LA',
        'cs': 'CS',
        'rv': 'RV'
    }
    for term, corrected_term in medical_terms.items():
        caption = caption.replace(term, corrected_term)
    # 6. Đảm bảo câu hoàn chỉnh với dấu chấm cuối
    caption = caption.strip()
    if caption and not caption.endswith('.'):
        caption += '.'

    return caption
# Đọc file run.csv
run_df = pd.read_csv('/kaggle/working/run.csv')

# Áp dụng hàm clean_caption cho cột Caption
run_df['Caption'] = run_df['Caption'].apply(clean_caption)

# Lưu lại file run.csv đã được làm sạch
run_df.to_csv('/kaggle/working/run_clean.csv', index=False)
print("Đã làm sạch và chuẩn hóa file run_clean.csv")

In [ ]:
# import shutil
# shutil.make_archive('run.csv','zip','/kaggle/working')

In [ ]:
# from IPython.display import FileLink
# FileLink(r'run.csv.zip')

In [ ]:
# import shutil
# shutil.make_archive('valid.csv','zip','/kaggle/working')

In [ ]:
# from IPython.display import FileLink
# FileLink(r'valid.csv.zip')